## David Perryman - Jan 6

## Setup

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
wavelen=3500
test_size = int(1e4)
energy=1000
osc_sizes = [0., 0.1*energy,0.2*energy,0.3*energy,0.4*energy]
noise_rms = 20.
noise = random.normal(loc=0., scale=noise_rms, size=(5,test_size,wavelen))
phases = random.random_sample((5,test_size))
phases = phases*2*pi
freq = 1./(wavelen)

### Porting Aaron's pulsegen code over to Python

In [3]:
def Box_Muller():
    return random.normal(loc=0., scale=1)

In [4]:
def gen_pulse(T0, amp, random=True):
    wf = empty(wavelen)
    # added this in to generate const. rise time pulsees
    if random:
        cc_slow = 2.5+0.4*Box_Muller(); cc_slow=cc_slow/(cc_slow+1); #charge collection slow time constant
    else:
        cc_slow = 2.5; cc_slow=cc_slow/(cc_slow+1); #charge collection slow time constant
        
    cc_fast = 1./2.5; #charge collection fast time constant
    alpha_cr = 1250./(1250.+1.); #fall time of output
    alpha_rc1 = 1./2.75;
    alpha_rc2 = 1./2.75;
#    step[2]={0,0},charge[2]={0,0},cur_s[2]={0,0},cur_f[2]={0,0},cr[2]={0,0},rc1[2]={0,0},rc2[2]={0,0};
    step=zeros(2);charge=zeros(2);cur_s=zeros(2);cur_f=zeros(2);cr=zeros(2);rc1=zeros(2);rc2=zeros(2);

    #T0=1000;#1000+2.5*Box_Muller();
    for i in range(wavelen):
        if (i>=T0):
            step[i%2]= 1.
        else:
            step[i%2]= 0.;
            
        cur_s[i%2]=cc_slow*(cur_s[(i+1)%2]+step[i%2]-step[(i+1)%2]);
        cur_f[i%2]=cc_fast*(cur_s[i%2]-cur_f[(i+1)%2])+cur_f[(i+1)%2];
        charge[i%2]=charge[(i+1)%2]+amp*cur_f[i%2]*(1./cc_slow-1.);
        cr[i%2]=alpha_cr*(cr[(i+1)%2]+charge[i%2]-charge[(i+1)%2]);
        rc1[i%2]=alpha_rc1*(cr[i%2]-rc1[(i+1)%2])+rc1[(i+1)%2];
        rc2[i%2]=alpha_rc2*(rc1[i%2]-rc2[(i+1)%2])+rc2[(i+1)%2];
        wf[i]=rc2[i%2];
    return wf

In [5]:
def fft_trapezoid(length, rise_time, flat_top , tau):
    wavelen = length;
    length = length+2*rise_time+flat_top;
    p = zeros(length)
    s = zeros(length)
    input2 = zeros(length)
    p[0] = s[0] = input2[0] = 0.;
    for i in range(1,length):
        input2[i] = s[i] = 0.;
    input2[1]=1.;
    tau = 1/(exp(1./tau)-1);
    for i in range(1,length):
        if i>=2*rise_time+flat_top:
            d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]+input2[i-2*rise_time-flat_top]
        else:
            if i>=rise_time+flat_top:
                d = input2[i]-input2[i-rise_time]-input2[i-rise_time-flat_top]
            else:
                if i>=rise_time:
                    d = input2[i]-input2[i-rise_time]
                else:
                    d = input2[i];
        p[i] = p[i-1]+d;
        s[i] = s[i-1]+p[i]+tau*d;
    for i in range(length):
        s[i] = s[i]/(rise_time*tau);
    
    res = fft.rfft(s)
    return res[:-100]


In [6]:
def find_max(data):
    max_val=float('-inf')
    max_i = 0.
    for i,x in enumerate(data):
        if x>max_val:
            max_i = i
            max_val = x
    return max_val, max_i

In [7]:
def find_T0(data):
    trape = fft_trapezoid(3500, 100, 1 , 1250.)
    trap_ = (fft.irfft(trape*(fft.rfft(data))))
    #plot(trap_)
    m,i = find_max(trap_)
    return i-101

In [8]:
def LS(wf, T0, freq):
    # removes an oscillation and returns the energy of the waveform
    A = zeros((3,wavelen))
    A[0] = gen_pulse(T0,1.)
    A[1] = sin(freq*t)
    A[2] = cos(freq*t)
    wf = matrix(wf).T
    A = matrix(A).T
    x = inv(A.T*A)*A.T*wf
    M = eye(wavelen) - A*inv(A.T*A)*A.T
    s2 = wf.T*M*wf/(wavelen-3)
    errors = []
    s2 = array(s2).flatten()[0]
    for i in range(3):
        errors.append(sqrt(s2*(array(inv(A.T*A))[i][i])))
    return append(array(x).T.flatten()[0], errors[0])

    

### Generating Data

In [ ]:
fixedT0Data = empty((5,test_size,wavelen))
varT0Data = empty((5,test_size,wavelen))
t = arange(3500)

for j,osc_amp in enumerate(osc_sizes):
    for i in range(test_size):
        fixedT0Data[j][i] = gen_pulse(1000,energy) + noise[j][i] + osc_amp*sin(phases[j][i]+freq*t)
        varT0Data[j][i] = gen_pulse(random.randint(900,1101),energy) + noise[j][i] + osc_amp*sin(phases[j][i]+freq*t)

In [ ]:
plot(fixedT0Data[4][1])

In [ ]:
plot(varT0Data[0][10])

## Oscillation Amplitude Has No Effect On Fit Accuracy

In [ ]:
results1 = empty((5,test_size,2))
for i in range(5):
    for j in range(test_size):
        results1[i][j] = LS(fixedT0Data[i][j], 1000, freq)

In [ ]:
fig = figure(figsize=(18,5))

subplot(121)
for i in range(5):
    hist(results1[i][:,0], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[i]));
title('Results of Energy Fit')
ylabel('Count')
xlabel('Energy Returned')
legend()

subplot(122)
for i in range(5):
    hist(results1[i][:,1], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[i]));
title('Standard Errors of Energy Fit')
ylabel('Count')
xlabel('Standar Error of Energy')
legend();

savefig('res1.png')

## Having to Find $T_0$ Does Not Significantly Increase Standard Error 

In [ ]:
results2 = empty((5,test_size,2))
for i in range(5):
    for j in range(test_size):
        fit_t0 = find_T0(varT0Data[i][j])
        results2[i][j] = LS(varT0Data[i][j], fit_t0, freq)

In [ ]:
fig = figure(figsize=(18,5))

subplot(121)
for i in range(5):
    hist(results2[i][:,0], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[i]));
title('Results of Energy Fit')
ylabel('Count')
xlabel('Energy Returned')
legend()

subplot(122)
for i in range(5):
    hist(results2[i][:,1], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[i]));
title('Standard Errors of Energy Fit')
ylabel('Count')
xlabel('Standard Error of Energy')
legend();

savefig('res2.png')

## Overshooting and Undershooting the Frequency Does Not Introduce Bias

In [ ]:
freq_offsets = array([[1.,1.01,1.05,1.1],[1.,0.99,0.95,0.9]])
freq_offsets

In [ ]:
results3 = empty((2,4,5,test_size,2))

for i in range(2):
    for j in range(4):
        for k in range(5):
            for l in range(test_size):
                fit_t0 = find_T0(varT0Data[k][l])
                results3[i][j][k][l] = LS(varT0Data[k][l], fit_t0, freq*freq_offsets[i][j])

In [ ]:
fig = figure(figsize=(18,24))

for i in range(4):
    subplot(8,2,i*2+1)
    for j in range(5):
        hist(results3[0][i][j][:,0], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[j]));
    title('Energy From OverShooting by '+str(round((freq_offsets[0][i]-1.)*100))+' percent')
    ylabel('Count')
    xlabel('Energy Returned')
    legend()

for i in range(4):
    subplot(8,2,(i+1)*2)
    for j in range(5):
        hist(results3[0][i][j][:,1], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[j]));
    title('Standard Error From OverShooting by '+str(round((freq_offsets[0][i]-1.)*100))+' percent')
    ylabel('Count')
    xlabel('Standard Error')
    legend()

subplots_adjust(hspace=1.)
savefig('res3.png')

In [ ]:
fig = figure(figsize=(20,30))

for i in range(4):
    subplot(8,2,i*2+1)
    for j in range(5):
        hist(results3[1][i][j][:,0], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[j]));
    title('Energy From underShooting by '+str(round(abs((freq_offsets[0][i]-1.)*100)))+' percent')
    ylabel('Count')
    xlabel('Energy Returned')
    legend()

for i in range(4):
    subplot(8,2,(i+1)*2)
    for j in range(5):
        hist(results3[1][i][j][:,1], bins=int(1e2), histtype='step', label='Osc. Size: '+str(osc_sizes[j]));
    title('Standard Error From UnderShooting by '+str(round(abs((freq_offsets[0][i]-1.)*100)))+' percent')
    ylabel('Count')
    xlabel('Standard Error')
    legend()

subplots_adjust(hspace=1.)
savefig('res4.png')

### Cool Matrices

All the cool kids know that standard error is an estimate of the standard deviation and that the variance of a population is described by $\Large {\frac{\sum_{i=0}^{N} y_i^2}{N}}$ for a sample of measurements $y_0,y_1,...,y_N$.

But what if I told you that you could find estimate this using matrices and your population sample??

If the data is approximated by $$ 
A x = y \rightarrow 
 \begin{pmatrix}
    \vdots & \vdots & \vdots \\
    f_{1} & f_2 & f_3 \\
    \vdots & \vdots & \vdots \\
\end{pmatrix} 
  \begin {pmatrix}
  x_{1} \\
  x_{2}\\
  x_{3} 
\end {pmatrix} = 
  \begin {pmatrix}
  y_{1} \\
  y_{2}\\
  \vdots\\
  y_{m} 
\end {pmatrix}
$$ where $f_1$ is the designed pulse shape, $f_2$ is a sine curve and $f_3$ is a cosine.

Then I could say that $ x=(A^{T} A)^{-1} A^{T} y$, and we could call the covariance matrix $ Q=(A^{T}A)^{-1}$. I can then phrase $ {\frac{\sum_{i=0}^{N} x_i^2}{N}}$ as $ (y - \hat y)^T (y-\hat y)$, which can be rearanged to yield $ \hat \sigma^2 = \large \frac{y(I - A (A^T A)^{-1} A^T)y}{N-d.o.f.}$

This means that the estimate of the error in $f_1$ is $ \sqrt {\large \frac{y(I - A (A^T A)^{-1} A^T)y}{N-d.o.f.} Q_{1,1}}$